# NIRCam Observer Example

This notebook will show how to run the rogue path tool on a NIRCam program to get the [position angle](https://jwst-docs.stsci.edu/jwst-observatory-characteristics-and-performance/jwst-position-angles-ranges-and-offsets#gsc.tab=0) (PA) ranges free from [claws](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/nircam-known-issues/nircam-scattered-light-artifacts#NIRCamScatteredLightArtifacts-clawsClaws). It will also show how to incorporate these PA ranges into your APT program to ensure the observations are only taken within those PA ranges.

## Table of Contents
1. [Imports](#1.-Imports)
2. [Input Data](#2.-Input-Data)
3. [Run the Tool](#3.-Run-the-Tool)
4. [Apply PA constraints to APT](#4.-Apply-PA-constraints-to-APT)
5. [Other Use Cases](#5.-Other-Use-Cases)

## 1. Imports

In [ ]:
import os
import pathlib

from jwst_rogue_path_tool.detect_claws import aptProgram
from jwst_rogue_path_tool.plotting import create_v3pa_vs_flux_plot
from jwst_rogue_path_tool.constants import PROJECT_DIRNAME

## 2. Input Data

The tool requires the [APT records file](https://jwst-docs.stsci.edu/jwst-astronomers-proposal-tool-overview/additional-jwst-apt-functionality/apt-export-files#APTExportFiles-records_filerecordsfile) as input, which is a json file that contains various information on the pointing and instrument configuration for each observation in the program. This file can be obtained by opening the program in APT then `File->Export->records.json file`. For this notebook, we'll simply use the example records file already contained within this repository `APT_test_4RPtool.records.json`.

In [ ]:
apt_json_filename = pathlib.Path(PROJECT_DIRNAME) / "data" / "APT_test_4RPtool.records.json"

## 3. Run the Tool

Next, we'll run the tool on the example program to get the predicted claw flux at each possible position angle. We'll also write out the position angles that are free from claws into a text file. We'll save both of these outputs in the current working directory; the claw flux plots and text files will be saved in the `v3pa_vs_flux` and `valid_angle_reports` subdirectories, respectively. The below cell will run on all observations which will take ~5 minutes; to save time, you can also run all of these steps on a single observation of interest, as demonstrated in the next cell.

In [ ]:
# Run the tool
program = aptProgram(apt_json_filename, angular_step=1.0)
program.run()

# Get supported observations (i.e. omit non-imaging NIRCam modes or other instruments)
supported_observations = program.observations.supported_observations.keys()

# For each observation, plot the predicted claw flux at each PA, and generate a text file showing the position angles free from claws
for obs_number in supported_observations:
    # Make plot of claws flux vs position angle
    create_v3pa_vs_flux_plot(program.observations.data[obs_number], output_directory=pathlib.Path(os.getcwd()))
    # Make file containing good position angles
    program.make_background_report(program.observations.data[obs_number], output_directory=os.getcwd())

Below, we show how to run the tool and get the outputs for a single observation. This time, we'll display the claw flux plots inline.

In [ ]:
# The observation number of interest
obs_number = 5

# Run the tool
program = aptProgram(apt_json_filename, angular_step=1.0, usr_defined_obs=[obs_number])
program.run()

# Make plot of claws flux vs position angle
create_v3pa_vs_flux_plot(program.observations.data[obs_number])
# Make file containing good position angles
program.make_background_report(program.observations.data[obs_number], output_directory=os.getcwd())

The plot above illustrates the predicted claw flux at each position angle for every filter. The horizontal orange and green lines indicate signal levels at 10% (based on minimum background throughout the year) and 20% (based on mean background throughout the year) of the predicted background levels, respectively. These thresholds help identify position angles free from claw contamination. Typically, the 20% threshold is sufficient, but for certain targets—such as faint, high-redshift objects—observers may prefer the more conservative 10% threshold. The PA ranges that are free from claws based on each threshold are saved to a text file, e.g. for this example observation `program_0_observation_5_background_module_B.txt`:
```
**** Ranges Not Impacted by Background Thresholds ****
**** Module B ****
**** Ranges Under 0.1 of min  ****
PA Start -- PA End: 0 -- 80
PA Start -- PA End: 103 -- 115
PA Start -- PA End: 126 -- 157
PA Start -- PA End: 162 -- 163
PA Start -- PA End: 184 -- 221
PA Start -- PA End: 243 -- 253
PA Start -- PA End: 263 -- 265
PA Start -- PA End: 270 -- 300
PA Start -- PA End: 310 -- 312
PA Start -- PA End: 315 -- 359
**** Ranges Under 0.2 of mean  ****
PA Start -- PA End: 0 -- 81
PA Start -- PA End: 92 -- 117
PA Start -- PA End: 124 -- 165
PA Start -- PA End: 178 -- 178
PA Start -- PA End: 183 -- 223
PA Start -- PA End: 235 -- 237
PA Start -- PA End: 240 -- 302
PA Start -- PA End: 308 -- 359
```

The PA ranges in this file are determined conservatively by considering all filters together. If the predicted claw flux in any filter exceeds the threshold, the corresponding position angle is excluded from these ranges.

## 4. Apply PA constraints to APT

Once the PA ranges free from claws are determined, users can add them to their APT files as a [PA Special Requirement](https://jwst-docs.stsci.edu/jppom/special-requirements/aperture-position-angle-special-requirements#gsc.tab=0) to ensure their observations are only taken within those PA ranges. To do this, select the observation in APT, then go to `Special Requirement→Add→Position Angle→PA Range` and input each good PA range in the V3PA Range boxes. Each PA range should be entered as a separate Special Requirement. For example, using the 20% threshold, the first Special Requirement for the example observation above would be:
```
Aperture PA Range 359.92542306 to 80.92542306 Degrees (V3 0.0 to 81.0)
```

Currently, the tool does not take into account target visibility, so some of the PA ranges shown may not need to be considered. For example, if a strong claw is predicted at a PA that is never available in the [roll analysis](https://jwst-docs.stsci.edu/jwst-astronomers-proposal-tool-overview/apt-workflow-articles/apt-visit-planner#gsc.tab=0), users don't have to include Special Requirements to avoid that PA as it will never be used anyway. To check which PA ranges are actually available for a given visit, users can go to `Visit Planner→Reports→Visitx.x→Total Roll Analysis For Visit`.

## 5. Other Use Cases

In this section, we demonstrate additional functionality that may be useful for observers.

The predicted claw flux for each filter and module can be accessed separately for a given observation. This allows for customized analysis, such as identifying good PA ranges based on custom thresholds or focusing on claw contamination for a particular filter or module.

Below, we show how to retrieve the predicted claw flux for a given filter and module and determine PA ranges where the predicted claw flux remains below a custom threshold.

In [ ]:
obs_number = 5  # the observation of interest
threshold = 50.0  # claw flux threshold in DN/ks
flux = program.observations.data[obs_number]['flux']['dn_pix_ks']['dn_pix_ks_CLEAR+F150W_B']  # predicted claw flux for obs 5 F150W module B
v3pa = np.arange(360)

# Get v3pa ranges where claw flux is below the chosen threshold
below_threshold = np.where(flux < threshold)[0]
ranges = np.split(v3pa[below_threshold], np.where(np.diff(below_threshold) != 1)[0] + 1)
print(f'**** Ranges Under {threshold} DN/ks ****')
for r in ranges:
    print(f'PA Start -- PA End: {r[0]} -- {r[-1]}')

# Plot the claw flux with shaded good v3pa ranges
plt.plot(flux)
plt.axhline(threshold, color='black', ls='--')
plt.yscale('log')
plt.ylabel('DN/pix/ks')
plt.xlabel('V3PA')
for r in ranges:
    plt.axvspan(r[0], r[-1], color='gray', alpha=0.2)